In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

path = '/kaggle/input/titanic/'
# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
import math

In [ ]:
df = pd.read_csv(os.path.join(path,'train.csv'))
df_test = pd.read_csv(os.path.join(path,'test.csv'))

In [ ]:
df.head()

In [ ]:
df_test.head()

In [ ]:
# df = df.append(df_test)

In [ ]:
# df.head()

In [ ]:
df = df.drop(columns=['Name', 'Ticket'])
df_test = df_test.drop(columns=['Name', 'Ticket'])

In [ ]:
df.describe()

In [ ]:
male_survived = len(df.loc[(df['Sex']=='male') & (df['Survived']==1)])
male_died = len(df.loc[(df['Sex']=='male') & (df['Survived']==0)])
female_survived = len(df.loc[(df['Sex']=='female') & (df['Survived']==1)])
female_died = len(df.loc[(df['Sex']=='female') & (df['Survived']==0)])

print(male_survived, male_died, male_survived+male_died)
print(female_survived, female_died, female_survived+female_died)

survived = plt.bar([0,1],[male_survived,male_died])
died = plt.bar([0,1],[female_survived,female_died],bottom=[male_survived,male_died])
plt.xticks([0,1],['Surived','Died'])
plt.legend((survived[0], died[0]), ('Men', 'Women'))
plt.show()

# Neat, I probably would've died

In [ ]:
# plt.scatter(df.loc[df['Survived']==1]['Age'], df.loc[df['Survived']==1]['Embarked'])

In [ ]:
plt.hist([df.loc[df['Survived']==1]['Age'], df.loc[df['Survived']==0]['Age']], stacked=True)
plt.show()

In [ ]:
plt.scatter(df.loc[df['Survived']==1]['Age'], df.loc[df['Survived']==1]['Fare'], s=df.loc[df['Survived']==1]['Fare'], c='blue')
plt.scatter(df.loc[df['Survived']==0]['Age'], df.loc[df['Survived']==0]['Fare'], s=df.loc[df['Survived']==0]['Fare'], c='red')
plt.legend(['Survived', 'Died'])
plt.show()

In [ ]:
df.head()

In [ ]:
mean_age_by_class = [0,0,0]
counter = [0,0,0]
for i in range(len(df)):
    if not math.isnan(df.loc[i,'Age']) and not math.isnan(df.loc[i,'Pclass']):
        mean_age_by_class[df.loc[i,'Pclass']-1] += df.loc[i,'Age']
        counter[df.loc[i,'Pclass']-1] += 1
for i in range(3):
    mean_age_by_class[i] /= counter[i]
    
for i in range(len(df)):
    if math.isnan(df.loc[i,'Age']):
        df.loc[i,'Age'] = mean_age_by_class[df.loc[i,'Pclass']-1]
    if math.isnan(df.loc[i,'Pclass']):
        k = 0
        lowest = abs(df.loc[i,'Age']-mean_age_by_class[0])
        for j in range(1,3):
            if lowest > abs(df.loc[i,'Age']-mean_age_by_class[j]):
                k=j
                lowest = abs(df.loc[i,'Age']-mean_age_by_class[j])
        df.loc[i,'Pclass'] = k+1

In [ ]:
for i in range(len(df_test)):
    if math.isnan(df_test.loc[i,'Age']):
        df_test.loc[i,'Age'] = mean_age_by_class[df_test.loc[i,'Pclass']-1]
    if math.isnan(df_test.loc[i,'Pclass']):
        k = 0
        lowest = abs(df_test.loc[i,'Age']-mean_age_by_class[0])
        for j in range(1,3):
            if lowest > abs(df_test.loc[i,'Age']-mean_age_by_class[j]):
                k=j
                lowest = abs(df_test.loc[i,'Age']-mean_age_by_class[j])
        df_test.loc[i,'Pclass'] = k+1

In [ ]:
df[df.isna().any(axis=1)]

In [ ]:
# a = df.loc[df.isna().any(axis=1)].loc[:,['Cabin','Embarked']]
for i in range(len(df)):
    if type(df.loc[i,'Cabin']) == str:
        print(df.loc[i,'Cabin'], df.loc[i,'Embarked'])

In [ ]:
df = df.drop(columns=['Cabin'])
df_test = df_test.drop(columns=['Cabin'])

In [ ]:
df.head()

In [ ]:
sex_dummies = pd.get_dummies(df.loc[:,'Sex'], prefix='Sex')
if 'Sex' in df.columns:
    df = df.drop(columns=['Sex'])
df = pd.concat([df, sex_dummies], axis=1)
df.head()

sex_dummies_test = pd.get_dummies(df_test.loc[:,'Sex'], prefix='Sex')
if 'Sex' in df_test.columns:
    df_test = df_test.drop(columns=['Sex'])
df_test = pd.concat([df_test, sex_dummies_test], axis=1)

In [ ]:
for i in range(len(df)):
    if type(df.loc[i,'Embarked']) != str:
        print(df.loc[i,:])

In [ ]:
s = df.loc[df['Embarked']=='S']['Fare'].mean()
c = df.loc[df['Embarked']=='C']['Fare'].mean()
q = df.loc[df['Embarked']=='Q']['Fare'].mean()
arr = [s,c,q]
lookup = ['S','C','Q']

for i in range(len(df)):
    if type(df.loc[i,'Embarked']) != str:
        k = 0
        lowest = abs(arr[0]-df.loc[0,'Fare'])
        for j in range(1,3):
            if lowest > abs(arr[j]-df.loc[i,'Fare']):
                k = j
                lowest = abs(arr[j]-df.loc[i,'Fare'])
        df.loc[i,'Embarked'] = lookup[k]

for i in range(len(df_test)):
    if type(df_test.loc[i,'Embarked']) != str:
        k = 0
        lowest = abs(arr[0]-df_test.loc[0,'Fare'])
        for j in range(1,3):
            if lowest > abs(arr[j]-df_test.loc[i,'Fare']):
                k = j
                lowest = abs(arr[j]-df_test.loc[i,'Fare'])
        df_test.loc[i,'Embarked'] = lookup[k]

for i in range(len(df_test)):
        if math.isnan(df_test.loc[i,'Fare']):
            df_test.loc[i,'Fare'] = arr[lookup.index(df_test.loc[i,'Embarked'])]

In [ ]:
embarked_dummies = pd.get_dummies(df.loc[:,'Embarked'], prefix='Embarked')
if 'Embarked' in df.columns:
    df = df.drop(columns=['Embarked'])
df = pd.concat([df, embarked_dummies], axis=1)
# df.head()

embarked_dummies_test = pd.get_dummies(df_test.loc[:,'Embarked'], prefix='Embarked')
if 'Embarked' in df_test.columns:
    df_test = df_test.drop(columns=['Embarked'])
df_test = pd.concat([df_test, embarked_dummies_test], axis=1)

In [ ]:
df.head()

In [ ]:
df_test.head()

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, train_test_split

In [ ]:
X = df.drop(columns=['Survived', 'PassengerId'])
y = df.loc[:,'Survived']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [ ]:
model = SVC(gamma='scale')
score = cross_val_score(model, X_train, y_train, cv=200, scoring='accuracy', n_jobs=-1)
print(score)

In [ ]:
score.mean()

In [ ]:
model = SVC(gamma='scale')
model.fit(X, y)

In [ ]:
if 'PassengerId' in df_test.columns:
    test_data = df_test.drop(columns='PassengerId').copy()
test_data.isna().any()

In [ ]:
prediction = model.predict(test_data)

In [ ]:
df_sub = pd.DataFrame({
    'PassengerId': df_test['PassengerId'],
    'Survived': prediction
})

df_sub.to_csv('submission.csv', index=False)

In [ ]:
df_sub = pd.read_csv('submission.csv')
df_sub.head(30)

In [ ]:
!kaggle competitions submit -c titanic -f submission.csv -m 'Heyo!'

<a href="./submission.csv">Download File</a>